In [1]:
import xarray as xr
import pyproj
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import cmocean as cmo
import glob
from mpl_toolkits.axes_grid1 import make_axes_locatable
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import numpy as np
import copy

In [2]:
run = 'n011'

fnames = sorted(glob.glob(f'../data/ecefiles/{run}/{run}*.nc'))

year0 = np.array([int(fname[-27:-23]) for fname in fnames])
year1 = np.array([int(fname[-18:-14]) for fname in fnames])
nyears = sum(year1+1-year0)

basin = np.arange(0,5)
bnames = ['EAIS','WEDD','AMUN','ROSS','APEN']
bcol = ['tab:green','tab:blue','tab:red','tab:orange','tab:purple']

In [3]:
#Get time-independent variables
ds = xr.open_dataset('../data/ecefiles/areas.nc')
area = ds['O1t0.srf'].values;
ds.close()

ds = xr.open_dataset(fnames[0])
lat = ds['nav_lat'].values
lon = ds['nav_lon'].values
levmid = ds['olevel'].values
lev = ds['olevel_bounds'].values
time_bnds = ds['time_centered_bounds']
thick = ds['e3t'].values #Quasi-time-independent, treated as fixed
ds.close()
secs = (time_bnds[:,1]-time_bnds[:,0]).values / np.timedelta64(1, 's')

In [4]:
#Get weighted mask for basin averages
lons = np.repeat(lon[np.newaxis,:,:],len(levmid),axis=0)
lats = np.repeat(lat[np.newaxis,:,:],len(levmid),axis=0)
mask = np.repeat(np.zeros(lons.shape)[np.newaxis,:,:,:],len(basin),axis=0)
aweight = np.zeros(mask.shape)

for b,bb in enumerate(basin):
    mm = np.zeros(lons.shape)
    if b==0:
        #EAIS
        mm[((lons<173) & (lons>-10)) & (lats<-65) & (lats>-76)] = 1
        depp = 369
    if b==1:
        #WEDD
        mm[(lons>-65) & (lons<-10) & (lats<-72)] = 1
        depp = 420
    if b==2:
        #AMUN
        mm[(lons>-150) & (lons<-80) & (lats<-70)] = 1
        depp = 305
    if b==3:
        #ROSS
        mm[((lons>150) | (lons<-150)) & (lats<-76)] = 1
        depp = 312
    if b==4:
        #APEN
        mm[(lons>-66) & (lons<-56) & (lats>-70) & (lats<-65)] = 1
        mm[(lons>-80) & (lons<-65) & (lats>-75) & (lats<-70)] = 1
        depp = 420
        
    z0 = depp-100.
    i0 = np.argmax(lev[:,1]>z0)
    mm[:i0,:,:] = 0
    w0 = (lev[i0,1]-z0)/(lev[i0,1]-lev[i0,0])
    mm[i0,:,:] = w0*mm[i0,:,:]
    for j in range(0,lon.shape[0]):
        for i in range(0,lon.shape[1]):
            if np.nansum(thick[0,i0:,j,i]) == 0:
                continue
            z1 = depp+100.
            i1 = np.argmin(lev[:,1]<z1)
            w1 = (z1-lev[i1,0])/(lev[i1,1]-lev[i1,0])
            mm[i1,j,i] = w1*mm[i1,j,i]
            mm[i1+1:,j,i] = 0
    mask[b,:,:,:] = mm*np.where(np.isnan(thick[0,:,:,:]),0,thick[0,:,:,:])/200.
    aweight[b,:,:,:] = mask[b,:,:,:]*area[np.newaxis,:,:] 

In [ ]:
#Calculate basin-average annual time series
tbas = np.zeros((len(basin),nyears))
ttime = np.arange(nyears)
months = np.arange(0,12)

c = 0
for f,fname in enumerate(fnames):
    ds = xr.open_dataset(fname)
    time = ds['time_centered'].values
    temp = ds['thetao'].values
    ds.close()
    
    tb = np.zeros((len(basin)))
    ny = int(len(time)/12)
    for y in np.arange(0,ny):
        for m,mm in enumerate(months):
            for b,bb in enumerate(bnames):
                tbb = np.nansum(temp[m+12*y,:,:,:]*aweight[b,:,:,:])/np.nansum(aweight[b,:,:,:])
                tb[b] += tbb*secs[m]
        
        for b,bb in enumerate(bnames):
            tbas[b,c] = tb[b]/sum(secs)
        print(c,tbas[:,c])
        tb = np.zeros((len(basin)))
        c += 1


0 [ 0.25193966 -1.50577199  2.40433786  1.35673781  0.12984292]
1 [ 0.22354953 -1.47730729  2.48575935  1.63699365  0.21272299]
2 [ 0.23286715 -1.50706805  2.51479127  1.40220394  0.17239681]
3 [ 0.2593282  -1.5629772   2.52768134  1.29979571  0.09274458]
4 [ 0.21065907 -1.57909392  2.52167665  1.34952798  0.05312205]
5 [ 0.14990851 -1.51907911  2.49392156  1.6206352  -0.07516993]
6 [ 0.23728455 -1.45895526  2.43910805  1.74700525 -0.0858355 ]
7 [ 0.29520509 -1.4538988   2.46447814  1.36762624 -0.07749244]
8 [ 0.35617915 -1.43881187  2.42530183  1.4083121  -0.20101286]
9 [ 0.46814568 -1.34979145  2.37931505  1.41318327 -0.28297246]
10 [ 0.53020445 -1.21105221  2.309411    1.44342186 -0.2918838 ]
11 [ 0.59455589 -1.10461075  2.27937403  1.3618874  -0.18828993]
12 [ 0.57626801 -1.12925956  2.23732273  1.04710292 -0.15094726]
13 [ 0.54831404 -1.16284443  2.2236054   0.74909489 -0.21072927]
14 [ 0.48963065 -1.17029936  2.30088502  0.80604934 -0.13920162]
15 [ 0.42710361 -1.26168192  2.3328

In [ ]:
mpl.rcParams['lines.linewidth'] = 2.
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['axes.titlesize'] = 9
mpl.rcParams['axes.labelsize'] = 9
mpl.rcParams['legend.fontsize'] = 9
mpl.rcParams['figure.subplot.wspace'] = .1
mpl.rcParams['figure.subplot.left'] = .1
mpl.rcParams['figure.subplot.right'] = .99
mpl.rcParams['figure.figsize'] = (7,4)

In [ ]:
fig = plt.figure()

ax = fig.add_subplot(121)
for b,bb in enumerate(bnames):
    ax.plot(ttime,tbas[b,:],c=bcol[b],label=bb)

ax.set_title('a) New pre-industrial control',loc='left')
ax.set_ylabel('Subsurface ocean temperature [degC]')
ax.set_xlabel('Time [years]')
ax.grid(True)
ax.set_xlim([ttime[0],ttime[-1]])

ax = fig.add_subplot(122,projection=ccrs.SouthPolarStereo())
ax.set_extent([-180, 180, -80, -63], crs=ccrs.PlateCarree())
for b,bb in enumerate(bnames):
    im = ax.scatter(lon,lat,15*np.nansum(mask[b,:,:,:],axis=0),c=bcol[b],transform=ccrs.PlateCarree(),label=bb)
ax.coastlines(lw=.1)
ax.set_title('b) Horizontal extent of ocean sectors',loc='left')
ax.legend(bbox_to_anchor=(-.02,-.15),loc='lower left',ncol=5,markerscale=2,columnspacing=.05,handletextpad=-.5)

gl = ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=False)
gl.xlocator = mticker.FixedLocator(np.arange(-180,210,30))
gl.xformatter = LONGITUDE_FORMATTER

plt.savefig(f'../draftfigs/ts_temp_{run}.png',dpi=450)
plt.show()